# Búsqueda. A\*. Estructuras de datos

Antes de llevar a cabo la implementación del A\* es necesario tener el cuenta que estructuras de datos podemos utilizar para:
1. Representar los nodos del grafo de búsqueda.
2. Las colecciones internas del algoritmo como abiertos y cerrados.

La elección correcta de (1.) nos va a permitir que el algoritmos sea genérico y pueda funcionar para muchos problemas de búsqueda diferentes y la elección correcta de (2.) nos va a simplificar la implementación y va a hacer que sea más eficiente.



## Representación de los nodos del gráfo de búsqueda

Vamos a definir una clase nodo. Esta clase va a tener 4 atributos:
1. Estado: Va a ser el estado del nodo actual.
2. Padre: El nodo desde el que se llega al nodo actual. En el nodo inicial el padre va a ser None.
3. g: El coste desde el nodo inicial a el nodo actual. Es igual al g del padre + el coste desde el padre al actual.
4. f: El coste total del camino que va desde el nodo inicial a el final pasando por el nodo actual. Es igual a g + heurística(estado)


A parte de un constructor al que le pasamos estos 4 atributos y los métodos getEstado, getPadre, getG y getF se van a necesitar una serie de métodos extra que nos van a ayudar con la implementación de A\*:

- camino: Obtiene el camino (todos los estados) desde el nodo actual hasta el inicio. En un bucle va recuperando el padre y mientras el padre no sea None añade el estado a una lista que finalmente se devolverá como camino.


```Python
nA=Nodo("A",None,0,10)
nB=Nodo("B",nA,1,10)
nC=Nodo("C",nA,1,8)
nD=Nodo("D",nB,2,10)

nD.camino()

#Devuelve ["A","B","D"]
```

- \_\_repr\_\_: Es equivalente al \_\_str\_\_ pero funciona en contenedores de Nodos (listas, diccionarios) y como los nodos los vamos a tener dentro de Abiertos y Cerrados, es mejor implementar este método. Tendrá que devolver el estado y la f, para poder hacer una traza de lo que pasa en el interior del algoritmo A\*. Opcionalmente podría devolver más cosas, aunque dificultaría la comprensión.

- El método \_\_lt\_\_ y el método  \_\_eq\_\_ son necesarios para poder usar el tipo Nodo dentro de Colas de Prioridad (el primero) y  Diccionarios (el segundo). El primero devuelve True cuando el objeto es mejor que otro que se pase por parámetro y el segundo devuelve True si el objeto por parámetro es igual. (Ya os les doy).





In [3]:
'''
Completar esto

'''

class Nodo:
    
    def __init__(self,estado,padre,g,f):
        self.estado = estado
        self.padre = padre
        self.g = g
        self.f = f
        
    def getPadre(self):
        return self.padre
    
    def getEstado(self):
        return self.estado
    
    def getG(self):
        return self.g
    
    def getF(self):
        return self.f
    
    def camino(self):
        nodoActual = self
        camino = [self.estado,]
        while (nodoActual.padre is not None):
            nodoActual = nodoActual.padre
            camino.append(nodoActual.estado)
        return list(reversed(camino))
    
    def __repr__(self):
        return "Nodo(estado=\"{}\", f={})".format(self.estado, self.f)
    
    def __lt__(self, other):
        return self.f<other.f
        

    def __eq__(self, other):        
        return self.getEstado()==other.getEstado()
    

In [5]:
nA=Nodo("A",None,0,10)
nB=Nodo("B",nA,1,10)
nC=Nodo("C",nA,1,8)
nD=Nodo("D",nB,2,10)
print(nD.camino())
print(nC.camino())
print(nB.camino())
print(nA.camino())

['A', 'B', 'D']
['A', 'C']
['A', 'B']
['A']


## Las colecciones internas del algoritmo A\*

### Cerrados: 
Hay que guardar los nodos visitados en alguna parte y además hay que poder consultar si cuando volvemos a evaluar un determinado nodo, un nodo con su mismo estado ya había sido visitado pero con una **f** mayor.

En un set podemos guardar elementos y podemos consultar si un elemento está incluido o no.

En un diccionario en el que la clave sea el estado y el valor asociado sea el nodo completo. Podemos consultar si un estado ha sido visitado y como podemos recuperar la **f** (al estar dentro del nodo) podemos saber si ese nodo pasa de cerrados a abiertos o no.

Para Cerrados elegimos un diccionario. 

```Python

cerrados = {}
cerrados["A"]=nA
cerrados["B"]=nB
cerrados["C"]=nC
cerrados["D"]=nD

# aparece un nuevo nodo con D
nuevaD=Nodo("D",nC,3,9)

viejaD = cerrados["D"]

if nuevaD.getF() < viejaD.getF():
    # elimino de cerrados y meto en abiertos
```

### Abiertos

Queremos una estructura que esté ordenada por la **F** de los nodos, para poder ir recuperando siempre el de menor **F**. En Python tenemos **PriorityQueue** que es una estructura que ordena los elementos por el valor que digamos.

La clase **PriorityQueue** tiene los siguientes métodos que nos interesan:
- put: Inserta elementos.
- empty: Comprueba si está vacía.
- get: Recupera el primer elemento de la cola (y desaparece de la cola)






In [8]:
import queue as queue

prio_queue = queue.PriorityQueue()
prio_queue.put((2, 'Item 1'))
prio_queue.put((1, 'Item 2'))
prio_queue.put((1, 'Item 3'))
prio_queue.put((5, 'Item 4'))
prio_queue.put((2, 'Item 5'))

while not prio_queue.empty():
    item = prio_queue.get() # el elemento se consume, desaparece
    print(item)

(1, 'Item 2')
(1, 'Item 3')
(2, 'Item 1')
(2, 'Item 5')
(5, 'Item 4')


Esta estructura nos vale pero tenemos que extenderla.

Porque en un parte del algoritmo A\* queremos saber si un determinado estado ya había sido almacenado en abiertos y porque si ya había sido almacenado pero tiene una f mayor que el nuevo queremos actualizarlo.

Vamos a crear una nueva estructura usando PriorityQueue y añadiendo lo que nos falta:
- un método para saber si tenemos un determinado Nodo en Abiertos.
- un método para actualizar un Nodo, se necesitará cuando encontramos Nodo con otro padre y una F menor.

In [23]:
prio_queue.put((2, 'Item 1'))
prio_queue.put((1, 'Item 2'))

'''
el atributo queue es una lista que tiene tuplas con la prioridad 
y el valor almacenado en la cola.

Se puede acceder a esa lista para implementar los métodos que nos hacen falta
'''


prio_queue.queue

[(1, 'Item 2'), (2, 'Item 1')]

In [21]:
# Completar esto

class Abiertos():
    def __init__(self):
        self.colaPrioridad = queue.PriorityQueue()
    
    def put(self,nodo):
        self.colaPrioridad.put((nodo.getF(),nodo))
    
    # le llamamos pop porque es más facil saber que el elemento se consume
    def pop(self):
        return self.colaPrioridad.get()
    
    def empty(self):
        return self.colaPrioridad.empty();
    
    # implementar este
    def getNodo(self,estado):
        for i in range(len(self.colaPrioridad.queue)):
            if self.colaPrioridad.queue[i][1].getEstado() == estado:
                return self.colaPrioridad.queue[i][1]
        return None
        # busco el estado en colaPrioridad.queue y devuelvo todo el nodo
        # o devuelvo None si no estaba
    
    # implementar este
    def update(self,nodoViejo,nodoNuevo):
        for i in range(len(self.colaPrioridad.queue)):
            if self.colaPrioridad.queue[i][1] == nodoViejo:
                self.colaPrioridad.queue.remove(self.colaPrioridad.queue[i])
                self.put(nodoNuevo)
        # hago un colaPrioridad.queue.remove para quitar el viejo
        # y un self.colaPrioridad.put para meter el nuevo
        
    
    def __str__(self):
        return str(self.colaPrioridad.queue)
        

# Descomentar para probar

abiertos = Abiertos()
abiertos.put(Nodo("A",None,0,10))
abiertos.put(Nodo("B","A",1,10))
abiertos.put(Nodo("C","A",1,8))
abiertos.put(Nodo("D","B",2,10))


while not abiertos.empty():
    item = abiertos.pop() # el elemento se consume
    print(item)
    

abiertos.put(Nodo("A",None,0,10))
abiertos.put(Nodo("B",nA,1,10))
abiertos.put(Nodo("C",nA,1,8))
abiertos.put(Nodo("D",nB,2,10))

print("actualizo D")
nodoD = abiertos.getNodo("D")
print(nodoD)
abiertos.update(nodoD,Nodo("D",nA,2,8))
print("abiertos")

while not abiertos.empty():
    item = abiertos.pop() # el elemento se consume
    print(item)

(8, Nodo(estado="C", f=8))
(10, Nodo(estado="A", f=10))
(10, Nodo(estado="B", f=10))
(10, Nodo(estado="D", f=10))
actualizo D
Nodo(estado="D", f=10)
abiertos
(8, Nodo(estado="C", f=8))
(8, Nodo(estado="D", f=8))
(10, Nodo(estado="B", f=10))
(10, Nodo(estado="A", f=10))
